In [727]:
import os
import sys
from collections import defaultdict
import timeit
import time

import importlib
import abxportinf
import abxportinf._grouper
import abxportinf._optimize
import abxportinf._vectorizer
import abxportinf.busdef
import abxportinf.main 
import abxportinf.util
from abxportinf.busdef import BusDef

modules = [
    '_grouper',
    '_optimize',
    '_vectorizer',
    'busdef',
    'main', 
    'util',
]
importlib.reload(abxportinf)
for mod in modules:
    importlib.reload(getattr(abxportinf, mod))


In [449]:
block_root = '/ddr'
comp_json_path = os.path.join(block_root, 'ddr.json5')
ports = abxportinf.get_ports_from_json5(comp_json_path)
#abxportinf.get_bus_matches(None, None)
#abxportinf.get_bus_defs(None)

Warning ('regHADDR', 'PAR_REG_ADDR_WIDTH + 2') not correctly parsed
Warning ('phy_reg_addr', '-(PAR_REG_ADDR_WIDTH)') not correctly parsed


In [475]:
bus_spec_rootdir = '/bus-defs/specs'
                                
print('loading bus defs from specs')
bus_defs = []                
for root, dirs, fnames in os.walk(bus_spec_rootdir):
    #print((root, dirs, files))  
    for fname in fnames:             
        spec_path = os.path.join(root, fname)
        if BusDef.is_spec_bus_def(spec_path):     
            print('  - loading ', spec_path)
            bus_defs.extend(
                BusDef.bus_defs_from_spec(spec_path)
            )                
print('  - done')                                            
print('loaded {} bus definitions from specs'.format(len(bus_defs)))
print('  - total req ports', sum([bd.num_req_ports for bd in bus_defs]))
print('  - total opt ports', sum([bd.num_opt_ports for bd in bus_defs]))

loading bus defs from specs
  - loading  /bus-defs/specs/amba.com/AMBA2/AHB/r3p0_1/AHB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Target/r0p0_0/AHB5Target_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Initiator/r0p0_0/AHB5Initiator_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4Stream/r0p0_1/AXI4Stream_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_RO_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE-Lite/r0p0_0/ACE-Lite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/APB4/r0p0_0/APB4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ATB/r0p0_0/ATB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACP/r0p0_0/ACP_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE/r0p0_0/ACE_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/AHBLite/r2p0_0/AHBLite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/APB/r2p0_0/APB_rtl.json5


In [603]:
def get_bus_defs():
    #return list(bus_defs)
    return list(filter(
        lambda bd: bd.abstract_type.name == 'AXI4_rtl',
        bus_defs,
    ))

In [520]:
stime = time.time()

bus_defs_s = get_bus_defs()
print('num matching bus defs:', len(bus_defs_s))

bus_mappings = abxportinf.get_bus_matches(ports, bus_defs_s)

etime = time.time()
print('time elapsed: {}s'.format(etime-stime))

num matching bus defs: 34
0/296 ('scanen', 1, 1)
10/296 ('dfi_phy_wrlvl_cs2_n', 2, 1)
20/296 ('dfi_phy_rdlvl_cs0_n', 2, 1)
30/296 ('dfi_phy_rdlvl_gate_cs1_n', 2, 1)
40/296 ('dfi_lvl_pattern_2', 4, -1)
50/296 ('dfi_rdlvl_gate_en', 9, -1)
60/296 ('dfi_wrdata_cs_n_0', 2, -1)
70/296 ('dfi_rddata_cs_n_p1_1', 2, -1)
80/296 ('dfi_wrdata_cs_n_p1_2', 2, -1)
90/296 ('regHRDATA', 32, -1)
100/296 ('regHWRITE', 1, 1)
110/296 ('phy_reg_mask', 4, -1)
120/296 ('dfi_we_n_p1', 1, -1)
130/296 ('dfi_rddata_en_p1', 9, -1)
140/296 ('dfi_rddata_en', 9, -1)
150/296 ('dfi_lp_ack', 1, 1)
160/296 ('dfi_bg', 2, -1)
170/296 ('port_busy', 2, -1)
180/296 ('axi0_AWSIZE', 3, 1)
190/296 ('axi0_AW_PARITY_EN', 1, 1)
200/296 ('axi0_ARID', 12, 1)
210/296 ('axi0_AR_PARITY', 1, 1)
220/296 ('axi0_ARREADY', 1, -1)
230/296 ('axi1_ARESETn', 1, 1)
240/296 ('axi1_AWCOBUF', 1, 1)
250/296 ('axi1_WDATA', 256, 1)
260/296 ('axi1_ARBURST', 2, 1)
270/296 ('axi1_WREADY', 1, -1)
280/296 ('axi1_RDATA', 256, -1)
290/296 ('lp_ext_done', 1, -1

In [728]:
port_group = abxportinf.get_test_port_group(ports)

stime = time.time()

bus_defs_s = get_bus_defs()
print('num matching bus defs:', len(bus_defs_s))

fcosts = sorted([
    (
        abxportinf._optimize.get_mapping_fcost(port_group, bus_def), 
        abxportinf._optimize.map_ports_to_bus(port_group, bus_def), 
        bus_def,
    )    
    for bus_def in bus_defs_s
], key=lambda x:x[0])
for fcost, (cost, mapping, cost_func), bus_def in fcosts:
    print('fcost:{}, cost:{}, driver:{}, abstract:{}'.format(
        fcost, cost, bus_def.driver_type, bus_def.abstract_type))
    for (is_opt, cost), pp, bp in sorted(
        [((bp in set(bus_def.opt_ports), cost_func(pp, bp)), pp, bp) for pp, bp in mapping.items()],
        key=lambda x: x[0],
    ):
        print('  - {:15s}:{:15s} {}'.format(str(pp), str(bp), 'opt' if is_opt else 'req'))
etime = time.time()
print('time elapsed: {}s'.format(etime-stime))

num matching bus defs: 2
fcost:18(n:0;w:9;d:9), cost:266(n:228;w:22;d:16), driver:slave, abstract:{'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4_rtl', 'version': 'r0p0_0'}
  - ('axi0_AWLEN', 8, 1):('AWLEN', 8, 1) req
  - ('axi0_AWSIZE', 3, 1):('AWSIZE', 3, 1) req
  - ('axi0_AWBURST', 2, 1):('AWBURST', 2, 1) req
  - ('axi0_WVALID', 1, 1):('WVALID', 1, 1) req
  - ('axi0_BREADY', 1, 1):('BREADY', 1, 1) req
  - ('axi0_WDATA', 256, 1):('WDATA', None, 1) req
  - ('axi0_AWADDR', 37, 1):('AWADDR', None, 1) req
  - ('axi0_WSTRB', 32, 1):('WSTRB', None, 1) req
  - ('axi0_AWVALID', 1, 1):('ARVALID', 1, 1) req
  - ('axi0_WLAST', 1, 1):('WLAST', 1, 1) opt
  - ('axi0_ARESETn', 1, 1):('ARESETn', 1, 1) opt
  - ('axi0_AWQOS', 1, 1):('AWQOS', 4, 1) opt
  - ('axi0_ARQOS', 1, 1):('ARQOS', 4, 1) opt
  - ('axi0_AWPROT', 2, 1):('AWPROT', 3, 1) opt
  - ('axi0_AWID', 12, 1):('AWID', None, 1) opt
  - ('axi0_AWLOCK', 1, 1):('ARLOCK', 1, 1) opt
  - ('axi0_AWREGION', 2, 1):('AWREGION', 4, 1) opt
  - ('ax

In [709]:
fcost, (cost, mapping, cost_func), bus_def = fcosts[0]
from abxportinf._optimize import MatchCost

def COST(mapping, ports, busdef):
    umap_ports = set(ports) - set(mapping.keys())
    umap_busports = set(busdef.req_ports) - set(mapping.values())
    cost = MatchCost.zero()          
    cost += sum([cost_func(p1, p2) for p1, p2 in mapping.items()])
    nil_port = ('', None, None)          
    cost += sum([cost_func(nil_port, p) for p in umap_ports])
    cost += sum([cost_func(nil_port, p) for p in umap_busports])
    return cost
    
print(COST(mapping, port_group, bus_def))
print(cost)
#for p1, p2 in mapping.items():
#    continue
#    #print(cost_func(p1,p2))
#costs = [cost_func(p1, p2) for p1, p2 in mapping.items()]
#umap_ports = set(bus_def.req_ports) - set(mapping.values())
#nil_port = ('', None, None)
#print(len(umap_ports))
#print(sum([cost_func(nil_port, p) for p in umap_ports]))

266(n:228;w:22;d:16)
161.67857142857142(n:53.89285714285714;w:53.89285714285714;d:53.89285714285714)


In [523]:
print((len(bus_mappings)))
for i, ((cost, mapping), bus_def)  in enumerate(sorted(bus_mappings, key=lambda x:x[0][0])):
#for i, (cost, bus_def, mapping) in enumerate(sorted(bus_mappings)):
    #if i > 5:
    #    print('BREAKING')
    #    break
    #print(bus_def.req_ports)
    #print(mapping)
    print('bus      type:', bus_def.bus_type)
    print('abstract type:', bus_def.abstract_type)
    print('driver   type:', bus_def.driver_type)
    print('num req ports:', bus_def.num_req_ports)
    print('num opt ports:', bus_def.num_opt_ports)
    for is_opt, pp, bp in sorted(
        [(bp in set(bus_def.opt_ports), pp, bp) for pp, bp in mapping.items()]
    ):
        print('  - {:15s}:{:15s} {}'.format(str(pp), str(bp), 'opt' if is_opt else 'req'))
    

34
bus      type: {'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4', 'version': 'r0p0_0'}
abstract type: {'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4_RO_rtl', 'version': 'r0p0_0'}
driver   type: slave
num req ports: 10
num opt ports: 13
  - ('axi0_AWADDR', 37, 1):('ARADDR', None, 1) req
  - ('axi0_AWBURST', 2, 1):('ARBURST', 2, 1) req
  - ('axi0_AWLEN', 8, 1):('ARLEN', 8, 1) req
  - ('axi0_AWQOS', 1, 1):('ARREADY', 1, -1) req
  - ('axi0_AWSIZE', 3, 1):('ARSIZE', 3, 1) req
  - ('axi0_AWVALID', 1, 1):('ARVALID', 1, 1) req
  - ('axi0_BREADY', 1, 1):('RREADY', 1, 1) req
  - ('axi0_WDATA', 256, 1):('RDATA', None, -1) req
  - ('axi0_WLAST', 1, 1):('RLAST', 1, -1) req
  - ('axi0_WVALID', 1, 1):('RVALID', 1, -1) req
  - ('axi0_ACLK', 1, 1):('ACLK', 1, 1)  opt
  - ('axi0_ARESETn', 1, 1):('ARESETn', 1, 1) opt
  - ('axi0_ARQOS', 1, 1):('ARQOS', 4, 1) opt
  - ('axi0_AWALLSTRB', 1, 1):('ARUSER', None, 1) opt
  - ('axi0_AWAPCMD', 1, 1):('ARCACHE', 4, 1) opt
  - ('axi0_AWCOBUF', 1